Basic recommendation system highlighting top 3 most relevant youTube videos of C3AI channel based on a user query

In [8]:
import json
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import re
from dotenv import load_dotenv

In [2]:
load_dotenv()

with open("data/c3ai_videos.json", "r", encoding="utf-8") as f:
    data = json.load(f)

title_list = [item["title"] for item in data]

title_list = [re.sub(r'c3\s?ai', '', title, flags=re.IGNORECASE).strip()
                for title in title_list]

url_list = [item["url"] for item in data]

In [3]:
#Generating the vector store (only once)

# metadatas = [{"url": url} for url in url_list]

# embeddings = OpenAIEmbeddings()

# vectorstore = FAISS.from_texts(title_list, embeddings, metadatas=metadatas)

# vectorstore.save_local("data/c3ai_youtube_index")

In [4]:
vectorstore = FAISS.load_local(
    "data/c3ai_youtube_index", 
    embeddings, 
    allow_dangerous_deserialization=True
)

### Basic question

In [5]:
query = "Learn the latest agentic advances from the company"
docs = vectorstore.similarity_search(query, k=3)
print(f"Rank#1: {docs[0].page_content} - Link: {docs[0].metadata['url']}")
print(f"Rank#2: {docs[1].page_content} - Link: {docs[1].metadata['url']}")
print(f"Rank#3: {docs[2].page_content} - Link: {docs[2].metadata['url']}")

Rank#1: Empowering AI Application Developers with ’s Latest Agentic AI Developer Innovations​ - Link: https://www.youtube.com/watch?v=01rldii7b_Q
Rank#2: Agentic AI Optimizes Processes in the Medical Equipment Manufacturing Industry - Link: https://www.youtube.com/watch?v=RTKNdCUeNsk
Rank#3: Transform Enterprise Automation with AI Agents |  Agentic Process Automation - Overview - Link: https://www.youtube.com/watch?v=ZfUe1NwLri0


### Job offer linked recommendations

In [6]:
with open("data/c3ai_job_offer.txt", "r", encoding="utf-8") as f:
    text = f.read()

text = text.split("Qualifications", 1)[0]

In [7]:
query = "What are the recommended videos to learn for applying this offer?" + text
docs = vectorstore.similarity_search(query, k=3)
print(f"Rank#1: {docs[0].page_content} - Link: {docs[0].metadata['url']}")
print(f"Rank#2: {docs[1].page_content} - Link: {docs[1].metadata['url']}")
print(f"Rank#3: {docs[2].page_content} - Link: {docs[2].metadata['url']}")

Rank#1: C3.ai Employee Insights - Machine Learning - Link: https://www.youtube.com/watch?v=en_IL0pA2XQ
Rank#2: C3.ai Employee Insights - Forward Deployed Engineering Management - Link: https://www.youtube.com/watch?v=DPfpw956VXU
Manufacturing | C3 Transform 2025 - Link: https://www.youtube.com/watch?v=IH55DoIsYdw


### Natural Language Answer

In [21]:
context = "\n\n---\n\n".join(
    f"[video {i}] {d.page_content} : {d.metadata["url"]}" for i, d in enumerate(docs, 1)
)

prompt = f"""Answer the question using the following provided videos

Question: {query}

Context:
{context}
"""

# Example with LangChain ChatOpenAI (adjust to your setup)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
answer = llm.invoke(prompt).content

print(answer)

To prepare for the Solution Engineer position at C3 AI, the following videos are recommended for learning:

1. **C3.ai Employee Insights - Machine Learning**: This video will provide insights into machine learning, which is crucial for developing AI-driven applications on the C3 AI Platform. Understanding machine learning concepts will enhance your ability to engineer full-stack applications effectively.

   [Watch here](https://www.youtube.com/watch?v=en_IL0pA2XQ)

2. **C3.ai Employee Insights - Forward Deployed Engineering Management**: This video will give you an understanding of the engineering management aspect, which is important for mentoring junior team members and conducting design and development reviews. It will also help you grasp the collaborative nature of the role.

   [Watch here](https://www.youtube.com/watch?v=DPfpw956VXU)

3. **Mastering AI for Engineering and Manufacturing | C3 Transform 2025**: This video focuses on the application of AI in engineering and manufact